# Apophis Ephemeris Download


Agustín Vallejo Villegas

This notebook will look up the data for the Apophis trajectory in JPL's Horizons web service using the Astroquery library.

An extra column is added at the end: `delta (Rt)` which will have the distance from Earth's center to Apophis in Earth Radii.

Finally, the dataframe is saved to the Data folder as `ephemeris.xlsx`


### Documentación
+ Horizons Query: 
    + https://astroquery.readthedocs.io/en/latest/jplhorizons/jplhorizons.html


### Librerías

In [1]:
"LIBRERÍAS BÁSICAS"
import numpy as np
import pandas as pd

In [5]:
python="/Users/jorgezuluagacallejas/opt/anaconda3/bin/python" # In my mac
!$python -m pip install astroquery

     |████████████████████████████████| 4.5 MB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 832 kB 44.0 MB/s eta 0:00:01
You should consider upgrading via the '/Users/jorgezuluagacallejas/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [6]:
"ASTROPY"
from astroquery.jplhorizons import Horizons

from astropy.time import Time
from astropy.table import Table
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, get_sun
import astropy.units as u

## Efemérides

In [7]:
"UNIDAD ASTRONÓMICA"
AU = (1*u.au).to(u.m).value

"TIEMPOS"
# Se definen los tiempos desde que está a 12Rt, pasando por su máximo
# acercamiento en 6Rt y volviendo a 12Rt
t_ini = Time('2029-04-13 13:30:00',format='iso')
t_max = Time('2029-04-13 22:05:00',format='iso') #Máximo Acercamiento
# t_end = Time('2029-04-13 23:00:00',format='iso') #Final. Step de 5min
t_end = Time('2029-04-14 06:06:00',format='iso')

epochs = {'start':t_ini.value, 'stop':t_end.value, 'step':'5m'}


print("(Ignorar advertencia de ERFA si sale, es porque está muy a futuro)")

(Ignorar advertencia de ERFA si sale, es porque está muy a futuro)


/Users/jorgezuluagacallejas/opt/anaconda3/lib/python3.8/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/jorgezuluagacallejas/opt/anaconda3/lib/python3.8/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


In [8]:
"EFEMÉRIDES"
#Efemérides de Apofis entre los tiempos estipulados
apophis = Horizons(id='99942',epochs=epochs) 
ephemeris = apophis.ephemerides()
vectors = apophis.vectors()

#Coordenadas Ecuatoriales del Asteroide
ECUcoords = SkyCoord(ephemeris['RA'], ephemeris['DEC']
                     ,ephemeris['delta'], frame='gcrs')

Nt = np.shape(ephemeris)[0]
print(Nt)

200


In [9]:
ephemeris['delta (Rt)'] = ephemeris['delta']*u.au.to(u.R_earth)

In [10]:
# ephemeris.to_pandas().to_csv("../Data/ephemeris3.csv")
vectors.to_pandas().to_csv("../Data/vectors2.csv")

In [11]:
len(vectors)

200